Importeren van modules

In [1]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np

Connectie maken met de  SSMS database

In [2]:
DB = {'servername': r'MSI\SQLEXPRESS',
    'database': 'johari'}

ssms_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                     ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
ssms_cursor = ssms_conn.cursor()
ssms_cursor

Microsoft Access database: go_staff

In [11]:
DB = {'servername': r'MSI\SQLEXPRESS',
    'database': 'johari'}

ssms_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                     ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
ssms_cursor = ssms_conn.cursor()



# Maak verbinding met de Access-database
""" go_staff_conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=../data/raw/go_staff.accdb;'
)
go_staff_conn = pyodbc.connect(go_staff_conn_str)
 """

go_staff_conn = sqlite3.connect(r"D:\School\Jaar 2\Project Datawarehouse\Johari_Venster\week_2\sqlite\go_staff.sqlite")
go_sales_conn = sqlite3.connect(r"D:\School\Jaar 2\Project Datawarehouse\Johari_Venster\week_2\sqlite\go_sales.sqlite")


# Lijst met tabelnamen van de go_staff-database
go_staff_table_names = ['course', 'sales_branch', 'sales_staff', 'satisfaction', 'satisfaction_type', 'training']

# Dictionary om de dataframes op te slaan
go_staff = {}

def read_source_table(conn, table_name):
    query = f'SELECT * FROM {table_name}'
    df = conn.execute(query)
    result = pd.DataFrame(df.fetchall(), columns=[column[0] for column in df.description])
    return result

def read_source_tables(conn_sales, conn_country, table_names):
    dfs = {}
    for table_name in table_names:
        query = f'SELECT * FROM {table_name}'
        df = pd.read_sql_query(query, conn_sales)
        dfs[table_name] = df

    # Read country table from different database
    query = 'SELECT * FROM country'
    df_country = pd.read_sql_query(query, conn_country)
    dfs['country'] = df_country

    # Merge sales_staff with sales_branch
    result = pd.merge(dfs['sales_staff'], dfs['sales_branch'], how='inner', left_on='SALES_BRANCH_CODE', right_on='SALES_BRANCH_CODE')

    # Merge the result with country
    result = pd.merge(result, dfs['country'], how='inner', left_on='COUNTRY_CODE', right_on='COUNTRY_CODE')

    return result

def clean_target_table(conn, table_name):
    ssms_cursor.execute(f'DELETE FROM {table_name}')
    ssms_conn.commit()

def course_etl():
    # Extract
    course = read_source_table(go_staff_conn, 'course')

    # Clean Target
    clean_target_table(ssms_conn, 'COURSE')
    # Sla de gegevens binnen de course-tabel op in de SQL Server-database
    for index, row in course.iterrows():
        try:
            #Transform
            course_id = row['COURSE_CODE']
            course_description = row['COURSE_DESCRIPTION']   

            #Load
            query = f"INSERT INTO dbo.COURSE (course_id, course_description) VALUES ({course_id}, '{course_description}')"
            ssms_cursor.execute(query)
            ssms_conn.commit()

        except Exception as e:
            print(e)
            break

def satisfaction_type_etl():
    satisfaction_type = read_source_table(go_staff_conn, 'satisfaction_type')

    # Clean Target
    clean_target_table(ssms_conn, 'rsatisfaction_type')

    for index, row in satisfaction_type.iterrows():
        try:
            satisfaction_type_id = row['SATISFACTION_TYPE_CODE']
            satisfaction_description = row['SATISFACTION_TYPE_DESCRIPTION']   

            query = f"INSERT INTO dbo.rsatisfaction_type (satisfaction_type_id, satisfaction_description) VALUES ({satisfaction_type_id}, '{satisfaction_description}')"

            ssms_cursor.execute(query)
            ssms_conn.commit()

        except Exception as e:
            print(e)
            break


def sales_staff_etl():
    table_names = ['sales_staff', 'sales_branch']
    sales_staff = read_source_tables(go_staff_conn, go_sales_conn, table_names)

    # Clean Target
    clean_target_table(ssms_conn, 'rSALES_STAFF')

    for index, row in sales_staff.iterrows():
        try:
            sales_staff_id = row['SALES_STAFF_CODE']
            sales_staff_first_name = row['FIRST_NAME']
            sales_staff_last_name = row['LAST_NAME']
            sales_staff_phone = row['WORK_PHONE']
            sales_staff_email = row['EMAIL']
            sales_staff_fax = row['FAX']
            sales_staff_sales_branch_id = row['SALES_BRANCH_CODE']
            sales_staff_position_en = row['POSITION_EN']
            sales_staff_date_hired = row['DATE_HIRED'] 
            sales_staff_extension = row['EXTENSION']
            sales_staff_sales_branch_address1 = row['ADDRESS1']
            sales_staff_sales_branch_address2 = row['ADDRESS2']
            sales_staff_sales_branch_city = row['CITY']
            sales_staff_sales_branch_region = row['REGION']
            sales_staff_sales_branch_postal_zone = row['POSTAL_ZONE']
            sales_staff_country_id = row['COUNTRY_CODE']
            sales_staff_country_name = row['COUNTRY']
            sales_staff_country_language = row['LANGUAGE']
            sales_staff_country_currency = row['CURRENCY_NAME']

            query = """
                INSERT INTO rSALES_STAFF (
                    sales_staff_id, 
                    sales_staff_first_name, 
                    sales_staff_last_name, 
                    sales_staff_phone, 
                    sales_staff_email, 
                    sales_staff_fax, 
                    sales_staff_sales_branch_id, 
                    sales_staff_position_en, 
                    sales_staff_date_hired, 
                    sales_staff_extension, 
                    sales_staff_sales_branch_address1, 
                    sales_staff_sales_branch_address2, 
                    sales_staff_sales_branch_city, 
                    sales_staff_sales_branch_region, 
                    sales_staff_sales_branch_postal_zone, 
                    sales_staff_country_id, 
                    sales_staff_country_name, 
                    sales_staff_country_language, 
                    sales_staff_country_currency
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """

            params = (
                sales_staff_id, 
                sales_staff_first_name, 
                sales_staff_last_name, 
                sales_staff_phone, 
                sales_staff_email, 
                sales_staff_fax, 
                sales_staff_sales_branch_id, 
                sales_staff_position_en, 
                sales_staff_date_hired, 
                sales_staff_extension, 
                sales_staff_sales_branch_address1, 
                sales_staff_sales_branch_address2, 
                sales_staff_sales_branch_city, 
                sales_staff_sales_branch_region, 
                sales_staff_sales_branch_postal_zone, 
                sales_staff_country_id, 
                sales_staff_country_name, 
                sales_staff_country_language, 
                sales_staff_country_currency
            )

            
            ssms_cursor.execute(query, params)
            ssms_conn.commit()

        except Exception as e:
            print(e)
            break


course_etl()
satisfaction_type_etl()
sales_staff_etl()



ssms_conn.close()

Microsoft Access database: go_sales

In [ ]:
go_staff_conn = sqlite3.connect(r"D:\School\Jaar 2\Project Datawarehouse\Johari_Venster\week_2\sqlite\go_staff.sqlite")
go_sales_conn = sqlite3.connect(r"D:\School\Jaar 2\Project Datawarehouse\Johari_Venster\week_2\sqlite\go_sales.sqlite")

# Lijst met tabelnamen van de go_sales-database
go_sales_table_names = ['country', 'order_details', 'order_header', 'order_method', 'product', 'product_line', 'product_type', 'retailer_site', 'return_reason', 'returned_item', 'sales_branch', 'sales_staff', 'SALES_TARGETData']

# Dictionary om de dataframes op te slaan
go_sales = {}

def read_source_table(conn, table_name):
    query = f'SELECT * FROM {table_name}'
    df = conn.execute(query)
    result = pd.DataFrame(df.fetchall(), columns=[column[0] for column in df.description])
    return result

def read_source_tables(conn_sales, conn_country, table_names):
    dfs = {}
    for table_name in table_names:
        query = f'SELECT * FROM {table_name}'
        df = pd.read_sql_query(query, conn_sales)
        dfs[table_name] = df

    # Read country table from different database
    query = 'SELECT * FROM sales_staff'
    df_staff = pd.read_sql_query(query, go_sales_conn)
    dfs['sales_staff'] = df_staff

    # Merge sales_staff with sales_branch
    result = pd.merge(dfs['sales_staff'], dfs['sales_branch'], how='inner', left_on='SALES_BRANCH_CODE', right_on='SALES_BRANCH_CODE')

    # Merge the result with country
    result = pd.merge(result, dfs['country'], how='inner', left_on='COUNTRY_CODE', right_on='COUNTRY_CODE')

    return result

def clean_target_table(conn, table_name):
    ssms_cursor.execute(f'DELETE FROM {table_name}')
    ssms_conn.commit()

def sales_etl():
    # Extract
    #SALES_TARGETData = read_source_table(go_sales_conn, 'SALES_TARGETData')
    table_names = ['SALES_TARGETData', 'product', 'product_line', 'product_type']
    SALES_TARGETData = read_source_tables(go_staff_conn, go_sales_conn, table_names)
    # Clean Target
    clean_target_table(ssms_conn, 'SALES_TARGETData')
    # Sla de gegevens binnen de course-tabel op in de SQL Server-database
    for index, row in SALES_TARGETData.iterrows():
        try:
            #Transform
            params = (
                row['TARGETDATA_ID'],
                row['TARGET_PRODUCT_ID'],
                row['TARGET_STAFF_ID'],
                row['TARGET_SALES_YEAR'],
                row['TARGET_SALES_PERIOD'],
                row['TARGET_SALES_TOTAL_TARGET_PERIOD'],
                row['TARGET_SALES_TOTAL_TARGET_YEAR'],
                row['TARGET_SALES_COST'],
                row['TARGET_SALES_MARGIN'],
                row['TARGET_SALES_TARGET_PROFIT']
            ) 

            #Load
            query = f"INSERT INTO dbo.COURSE (
                TARGETDATA_ID, 
                TARGET_PRODUCT_ID,
                TARGET_STAFF_ID,
                TARGET_SALES_YEAR,
                TARGET_SALES_PERIOD,
                TARGET_SALES_TOTAL_TARGET_PERIOD,
                TARGET_SALES_TOTAL_TARGET_YEAR,
                TARGET_SALES_COST,
                TARGET_SALES_MARGIN,
                TARGET_SALES_TARGET_PROFIT
                ) 
                VALUES 
                (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
            ssms_cursor.execute(query, params)
            ssms_conn.commit()

        except Exception as e:
            print(e)
            break

Microsoft Access database: go_crm

In [8]:
DB = {'servername': r'MSI\SQLEXPRESS',
    'database': 'johari'}

ssms_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                     ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
ssms_cursor = ssms_conn.cursor()


go_crm_conn = sqlite3.connect(r"D:\School\Jaar 2\Project Datawarehouse\Johari_Venster\week_2\sqlite\go_crm.sqlite")
# Lijst met tabelnamen van de go_sales-database
go_crm_table_names = ['age_group', 'country', 'retailer', 'retailer_contact', 'retailer_headquarters', 'retailer_segment', 'retailer_site', 'retailer_type', 'sales_demographic', 'sales_territory']

# Dictionary om de dataframes op te slaan
go_crm = {}

def read_source_table(conn, table_name):
    query = f'SELECT * FROM {table_name}'
    df = conn.execute(query)
    result = pd.DataFrame(df.fetchall(), columns=[column[0] for column in df.description])
    return result

def read_source_tables(conn_crm, table_names):
    dfs = {}
    for table_name in table_names:
        query = f'SELECT * FROM {table_name}'
        df = pd.read_sql_query(query, conn_crm)
        if 'TRIAL222' in df.columns:
            df = df.rename(columns={'TRIAL222': f'TRIAL222_{table_name}'})
        dfs[table_name] = df

    result = pd.merge(dfs['retailer_contact'], dfs['retailer_site'], how='inner', left_on='RETAILER_SITE_CODE', right_on='RETAILER_SITE_CODE')

    result = pd.merge(result, dfs['retailer'], how='inner', left_on='RETAILER_CODE', right_on='RETAILER_CODE')
    result = pd.merge(result, dfs['country'], how='inner', left_on='COUNTRY_CODE', right_on='COUNTRY_CODE')
    result = pd.merge(result, dfs['retailer_headquarters'], how='inner', left_on='RETAILER_CODEMR', right_on='RETAILER_CODEMR')
    result = pd.merge(result, dfs['retailer_segment'], how='inner', left_on='SEGMENT_CODE', right_on='SEGMENT_CODE')
    result = pd.merge(result, dfs['retailer_type'], how='inner', left_on='RETAILER_TYPE_CODE', right_on='RETAILER_TYPE_CODE')
    result = pd.merge(result, dfs['sales_demographic'], how='inner', left_on='RETAILER_CODEMR', right_on='RETAILER_CODEMR')
    result = pd.merge(result, dfs['age_group'], how='inner', left_on='AGE_GROUP_CODE', right_on='AGE_GROUP_CODE')
    result = pd.merge(result, dfs['sales_territory'], how='inner', left_on='SALES_TERRITORY_CODE', right_on='SALES_TERRITORY_CODE')
    print(result.head() )

    testdb = sqlite3.connect(r"test.sqlite")
    result.to_sql('retailer_contact', testdb, if_exists='replace', index=False)
    return result

def clean_target_table(conn, table_name):
    ssms_cursor.execute(f'DELETE FROM {table_name}')
    ssms_conn.commit()

def retailer_contact_etl():
    retailer_contact = read_source_tables(go_crm_conn, ['age_group', 'country', 'retailer', 'retailer_contact', 'retailer_headquarters', 'retailer_segment', 'retailer_site', 'retailer_type', 'sales_demographic', 'sales_territory'])
    #retailer_contact.head()
    # Clean Target
    clean_target_table(ssms_conn, 'RETAILER_CONTACT')

    for index, row in retailer_contact.iterrows():
        try:
            
            # Check if the row already exists because fckung pk violation
            check_query = "SELECT COUNT(*) FROM RETAILER_CONTACT WHERE retailer_contact_contact_id = ?"
            ssms_cursor.execute(check_query, (row['RETAILER_CONTACT_CODE'],))
            count = ssms_cursor.fetchone()[0]

            # If the row doesn't exist, insert it
            if count == 0:
                query = """
                    INSERT INTO RETAILER_CONTACT (
                        retailer_contact_contact_id, 
                        retailer_contact_site_id, 
                        retailer_contact_first_name, 
                        retailer_contact_last_name, 
                        retailer_contact_job_position_en, 
                        retailer_contact_extension, 
                        retailer_contact_fax, 
                        retailer_contact_email, 
                        retailer_contact_gender, 
                        retailer_id, 
                        retailer_codemr, 
                        retailer_company_name, 

                        retailer_age_group_id, 
                        retailer_age_group_upper_age, 
                        retailer_age_group_lower_age, 
                        retailer_sales_demographic_id, 
                        retailer_sales_demographic_codemr, 
                        retailer_sales_demographic_age_group_id, 
                        retailer_sales_demographic_sales_percent, 
                        retailer_sales_territory_id, 
                        retailer_sales_territory_name_en, 
                        retailer_country_id, 
                        retailer_country_country_en, 
                        retailer_country_flag_image, 
                        retailer_country_sales_territory_id, 
                        retailer_site_id, 
                        retailer_site_retailer_id, 
                        retailer_site_address1, 
                        retailer_site_address2, 
                        retailer_site_city, 
                        retailer_site_region, 
                        retailer_site_postal_zone, 
                        retailer_site_country_id, 
                        retailer_site_active_indicator, 
                        retailer_headquarters_codemr, 
                        retailer_headquarters_name, 
                        retailer_headquarters_address1, 
                        retailer_headquarters_address2, 
                        retailer_headquarters_city, 
                        retailer_headquarters_region,  
                        retailer_headquarters_postal_zone, 
                        retailer_headquarters_country_id, 
                        retailer_headquarters_phone, 
                        retailer_headquarters_fax, 
                        retailer_headquarters_segment_id, 
                        retailer_type_id, 
                        retailer_type_type_en, 
                        retailer_segment_id, 
                        retailer_segment_language, 
                        retailer_segment_name, 
                        retailer_segment_description 
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    """
                
                params = (
                    row['RETAILER_CONTACT_CODE'],
                    row['RETAILER_SITE_CODE'],
                    row['FIRST_NAME'],
                    row['LAST_NAME'],
                    row['JOB_POSITION_EN'],
                    row['EXTENSION'],
                    row['FAX_x'],
                    row['E_MAIL'],
                    row['GENDER'],
                    row['RETAILER_CODE'],
                    row['RETAILER_CODEMR'],
                    row['COMPANY_NAME'],
                    #row['RETAILER_TYPE_CODE'],
                    row['AGE_GROUP_CODE'],
                    row['UPPER_AGE'],
                    row['LOWER_AGE'],
                    row['DEMOGRAPHIC_CODE'],
                    row['RETAILER_CODEMR'],
                    row['AGE_GROUP_CODE'],
                    row['SALES_PERCENT'],
                    row['SALES_TERRITORY_CODE'],
                    row['TERRITORY_NAME_EN'],
                    row['COUNTRY_CODE_x'],
                    row['COUNTRY_EN'],
                    row['FLAG_IMAGE'],
                    row['SALES_TERRITORY_CODE'],
                    row['RETAILER_SITE_CODE'],
                    row['RETAILER_CODE'],
                    row['ADDRESS1_x'],
                    row['ADDRESS2_x'],
                    row['CITY_x'],
                    row['REGION_x'],
                    row['POSTAL_ZONE_x'],
                    row['COUNTRY_CODE_x'],
                    row['ACTIVE_INDICATOR'],
                    row['RETAILER_CODEMR'],
                    row['RETAILER_NAME'],
                    row['ADDRESS1_y'],
                    row['ADDRESS2_y'],
                    row['CITY_y'],
                    row['REGION_y'],
                    row['POSTAL_ZONE_y'],
                    row['COUNTRY_CODE_y'],
                    row['PHONE'],
                    row['FAX_y'],
                    row['SEGMENT_CODE'],
                    row['RETAILER_TYPE_CODE'],
                    row['RETAILER_TYPE_EN'],
                    row['SEGMENT_CODE'],
                    row['LANGUAGE'],
                    row['SEGMENT_NAME'],
                    row['SEGMENT_DESCRIPTION']
                )
                ssms_cursor.execute(query, params)
                ssms_conn.commit()

        except Exception as e:
            print(e)
            break

retailer_contact_etl()

  RETAILER_CONTACT_CODE RETAILER_SITE_CODE FIRST_NAME LAST_NAME  \
0                    10                 14      Frank     Smith   
1                    10                 14      Frank     Smith   
2                    10                 14      Frank     Smith   
3                    10                 14      Frank     Smith   
4                    10                 14      Frank     Smith   

   JOB_POSITION_EN EXTENSION             FAX_x                   E_MAIL  \
0  Chief Purchaser      2489  1 (514) 741-1610  FSmith@legolfeurinc.com   
1  Chief Purchaser      2489  1 (514) 741-1610  FSmith@legolfeurinc.com   
2  Chief Purchaser      2489  1 (514) 741-1610  FSmith@legolfeurinc.com   
3  Chief Purchaser      2489  1 (514) 741-1610  FSmith@legolfeurinc.com   
4  Chief Purchaser      2489  1 (514) 741-1610  FSmith@legolfeurinc.com   

  GENDER TRIAL222_retailer_contact  ... TRIAL222_retailer_type  \
0      M                         T  ...                      T   
1      M      

Microsoft Excel bestand: GO_SALES_INVENTORY_LEVELSData

In [ ]:
GO_SALES_INVENTORY_LEVELSData = pd.read_csv('../data/raw/GO_SALES_INVENTORY_LEVELSData.csv')
GO_SALES_INVENTORY_LEVELSData.head()

Microsft Excel bestand: GO_SALES_PRODUCT_FORECASTData

In [ ]:
GO_SALES_PRODUCT_FORECASTData = pd.read_csv('../data/raw/GO_SALES_PRODUCT_FORECASTData.csv')
GO_SALES_PRODUCT_FORECASTData.head()

In [ ]:
product = pd.read_excel('../data/raw/product.xlsx')


# Columns in the product dataframe: PRODUCT_NUMBER	INTRODUCTION_DATE	PRODUCT_TYPE_CODE	PRODUCTION_COST	MARGIN	PRODUCT_IMAGE	LANGUAGE	PRODUCT_NAME	DESCRIPTION
# Columns in the SSMS Database: product_product_id	product_name	product_image	product_line_name	product_type_name	product_introduction_date	product_cost	product_margin	product_language

for index, row in product.iterrows():
    try:
        introduction_date = pd.to_datetime(row['INTRODUCTION_DATE']).strftime('%Y-%m-%d')
        query = f"""
        INSERT INTO dbo.rPRODUCT 
        (product_product_id, product_name, product_image, product_line_name, product_type_name, product_introduction_date, product_cost, product_margin, product_language) 
        VALUES 
        ({row['PRODUCT_NUMBER']}, '{row['PRODUCT_NAME']}', '{row['PRODUCT_IMAGE']}', '', '{row['PRODUCT_TYPE_CODE']}', '{introduction_date}', '{row['PRODUCTION_COST']}', {row['MARGIN']}, '{row['LANGUAGE']}')
        """
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

# View the data in the SSMS Database
query = "SELECT * FROM dbo.rPRODUCT"
ssms_cursor.execute(query)
rows = ssms_cursor.fetchall()
df = pd.DataFrame.from_records(rows, columns=[desc[0] for desc in ssms_cursor.description])

Aanmaken van een SQLite database. Hier worden alle bewerkingen toegepast. Pas wanneer alles klaar is, worden alle gegevens binnen deze SQLite database overgezet naar de SSMS database

In [ ]:
import datetime

# Voor het geval dat de database nog niet bestaat, maak een nieuwe SQLite-database aan
sqlite_conn = sqlite3.connect('../data/processed/johari_sqlite.db')

# Nieuwe dataframes met dummy data
dbo_product = pd.DataFrame({
    'PRODUCT_id': [1, 2, 3],
    'PRODUCT_name': ['Moondrop Variations', 'Sony IER-Z1R', 'Truthear Nova'],
    'PRODUCT_image': ['moondrop.jpg', 'sony.jpg', 'truthear.jpg'],
    'PRODUCT_LINE_name': ['Moondrop', 'Sony', 'Truthear'],
    'PRODUCT_TYPE_name': ['In-ear monitor', 'In-ear monitor', 'In-ear monitor'],
    'PRODUCT_INTRODUCTION_DATE_date': [datetime.date(2021, 1, 1), datetime.date(2021, 1, 1), datetime.date(2021, 1, 1)],
    'PRODUCT_COST_price': [400, 1100, 180],
    'PRODUCT_MARGIN_dimensions': [0.5, 0.6, 0.7],
    'PRODUCT_LANGUAGE_name': ['English', 'English', 'English'],
})

dbo_sales_staff = pd.DataFrame({
    'SALES_STAFF_id': [1, 2, 3],
    'SALES_STAFF_f_name': ['Karel', 'Henk', 'Regen'],
    'SALES_STAFF_l_name': ['Ankerwout', 'Tank', 'Achterstevoren'],
    'SALES_STAFF_phone': ['+316456789', '+316654321', '+316123123'],
    'SALES_STAFF_email': ['k.anker@yandex.ru', 'henk@yahoo.com', 'nee@ger.com'],
    'SALES_STAFF_fax': ['+316456789', '+316654321', '+316123123'],
    'SALES_STAFF_sales_branch_id': [1, 2, 3],
    'SALES_STAFF_POSITION_EN_name': ['Sales Manager', 'Sales Manager', 'Sales Manager'],
    'SALES_STAFF_DATE_HIRED_date': [datetime.date(2021, 1, 1), datetime.date(2021, 1, 1), datetime.date(2021, 1, 1)],
    'SALES_STAFF_EXTENSION_extension': ['+316456789', '+316654321', '+316123123'],
    'SALES_STAFF_SALES_BRANCH_address1': ['Kerkstraat 1', 'Kerkstraat 2', 'Kerkstraat 3'],
    'SALES_STAFF_SALES_BRANCH_address2': ['Kerkstraat 1', 'Kerkstraat 2', 'Kerkstraat 3'],
    'SALES_STAFF_SALES_BRANCH_region': ['Noord', 'Zuid', 'Oost'],
    'SALES_STAFF_SALES_BRANCH_city': ['Amsterdam', 'Rotterdam', 'Utrecht'],
    'SALES_STAFF_SALES_BRANCH_postal_zone': ['1000AA', '2000BB', '3000CC'],
    'SALES_STAFF_COUNTRY_id': [1, 2, 3],
    'SALES_STAFF_COUNTRY_name': ['Netherlands', 'Belgium', 'Germany'],
    'SALES_STAFF_COUNTRY_language': ['Dutch', 'French', 'German'],
    'SALES_STAFF_COUNTRY_currency': ['Euro', 'Euro', 'Euro'],
})

dbo_retailer_contact = pd.DataFrame({
    'retailer_contact_contact_id': np.random.randint(1, 100, size=10),
    'retailer_contact_site_id': np.random.randint(1, 100, size=10),
    'retailer_contact_first_name': np.random.choice(['John', 'Jane', 'Jim', 'Jill', 'Joe'], size=10),
    'retailer_contact_last_name': np.random.choice(['Smith', 'Johnson', 'Williams', 'Jones', 'Brown'], size=10),
    'retailer_contact_job_position_en': np.random.choice(['Manager', 'Employee', 'CEO', 'CFO', 'CTO'], size=10),
    'retailer_contact_extension': np.random.randint(1, 100, size=10),
    'retailer_contact_fax': np.random.randint(1, 100, size=10),
    'retailer_contact_email': [f'test{i}@test.com' for i in range(10)],
    'retailer_contact_gender': np.random.choice(['M', 'F'], size=10),
    'retailer_id': np.random.randint(1, 100, size=10),
    'retailer_codemr': np.random.randint(1, 100, size=10),
    'retailer_company_name': [f'Company{i}' for i in range(10)],
    'retailer_age_group_id': np.random.randint(1, 100, size=10),
    'retailer_age_group_upper_age': np.random.randint(30, 60, size=10),
    'retailer_age_group_lower_age': np.random.randint(20, 30, size=10),
    'retailer_sales_demographic_id': np.random.randint(1, 100, size=10),
    'retailer_sales_demographic_codemr': np.random.randint(1, 100, size=10),
    'retailer_sales_demographic_age_group_id': np.random.randint(1, 100, size=10),
    'retailer_sales_demographic_sales_percent': np.random.random(size=10),
    'retailer_sales_territory_id': np.random.randint(1, 100, size=10),
    'retailer_sales_territory_name_en': [f'Territory{i}' for i in range(10)],
    'retailer_country_id': np.random.randint(1, 100, size=10),
    'retailer_country_country_en': [f'Country{i}' for i in range(10)],
    'retailer_country_flag_image': [f'Image{i}' for i in range(10)],
    'retailer_country_sales_territory_id': np.random.randint(1, 100, size=10),
    'retailer_site_id': np.random.randint(1, 100, size=10),
    'retailer_site_retailer_id': np.random.randint(1, 100, size=10),
    'retailer_site_address1': [f'Address{i}' for i in range(10)],
    'retailer_site_address2': [f'Address{i}' for i in range(10)],
    'retailer_site_city': [f'City{i}' for i in range(10)],
    'retailer_site_region': [f'Region{i}' for i in range(10)],
    'retailer_site_postal_zone': [f'Postal{i}' for i in range(10)],
    'retailer_site_country_id': np.random.randint(1, 100, size=10),
    'retailer_site_active_indicator': np.random.choice([True, False], size=10),
    'retailer_headquarters_codemr': np.random.randint(1, 100, size=10),
    'retailer_headquarters_name': [f'Name{i}' for i in range(10)],
    'retailer_headquarters_address1': [f'Address{i}' for i in range(10)],
    'retailer_headquarters_address2': [f'Address{i}' for i in range(10)],
    'retailer_headquarters_city': [f'City{i}' for i in range(10)],
    'retailer_headquarters_region': [f'Region{i}' for i in range(10)],
    'retailer_headquarters_postal_zone': [f'Postal{i}' for i in range(10)],
    'retailer_headquarters_country_id': np.random.randint(1, 100, size=10),
    'retailer_headquarters_phone': [f'Phone{i}' for i in range(10)],
    'retailer_headquarters_fax': [f'Fax{i}' for i in range(10)],
    'retailer_headquarters_segment_id': np.random.randint(1, 100, size=10),
    'retailer_type_id': np.random.randint(1, 100, size=10),
    'retailer_type_type_en': [f'Type{i}' for i in range(10)],
    'retailer_segment_id': np.random.randint(1, 100, size=10),
    'retailer_segment_language': [f'Language{i}' for i in range(10)],
    'retailer_segment_name': [f'Segment Name{i}' for i in range(10)],
    'retailer_segment_description': [f'Segment Description{i}' for i in range(10)],
})

dbo_returned_reason = pd.DataFrame({
    'RETURNED_REASON_id': [1, 2, 3],
    'RETURNED_REASON_DESCRIPTION_name': ['Broken', 'Not satisfied', 'Wrong product'],
})

dbo_order_method = pd.DataFrame({
    'ORDER_METHOD_id': [1, 2, 3],
    'ORDER_METHOD_EN_name': ['Online', 'Phone', 'Mail'],
})

dbo_course = pd.DataFrame({
    'COURSE_id': [1, 2, 3],
    'COURSE_DESCRIPTION_en': ['Programming', 'Querying', 'Spreadsheets'],
})

dbo_satisfaction_type = pd.DataFrame({
    'SATISFACTION_TYPE_id': [1, 2, 3],
    'SATISFACTION_TYPE_DESCRIPTION_name': ['Good', 'Bad', 'Neutral'],
})


# Surrogate keys toevoegen aan de dataframes (plus beredeneringen)

# Wanneer deze dataframe samengevoegd wordt met externe bronnen, kan het zijn dat de PK niet meer uniek is of conflicts heeft.
dbo_product['PRODUCT_SK'] = list(range(1, len(dbo_product['PRODUCT_id']) + 1)) 

# Wanneer er herstructurering plaatsvindt in de organisatie, kan het zijn dat de PK niet meer uniek is of conflicts heeft.
dbo_sales_staff['SALES_STAFF_SK'] = list(range(1, len(dbo_sales_staff['SALES_STAFF_id']) + 1))

# Een contactpersoon kan van functie veranderen, maar het kan zijn dat je oude data wilt behouden voor bijvoorbeeld trendanalyses.
dbo_retailer_contact['retailer_contact_SK'] = list(range(1, len(dbo_retailer_contact['retailer_contact_contact_id']) + 1))

# Je kan in de loop van tijd de beredeneringen willen veranderen. Een voorbeeld hiervan is wanneer je een beschrijving wilt opsplitsen omdat het te vaag is, maar nog steeds de oorspronkelijke PK wilt gebruiken om aan te duiden dat de opgesplitste beschrijvingen bij elkaar horen. Dit mag natuurlijk niet, hierdoor wordt de SK nu gebruikt als de unieke kenmerk.
dbo_returned_reason['RETURNED_REASON_SK'] = list(range(1, len(dbo_returned_reason['RETURNED_REASON_id']) + 1))

# Hetzelfde als bij de vorige, maar in plaats van een beschrijving, kan het bijvoorbeeld zijn dat een order methode wordt opgesplitst (zoals Online = Webshop, App, etc.).
dbo_order_method['ORDER_METHOD_SK'] = list(range(1, len(dbo_order_method['ORDER_METHOD_id']) + 1))

# Hetzelfde als bij de vorige, maar in plaats van een beschrijving, kan het bijvoorbeeld zijn dat een course wordt opgesplitst (zoals Programming = Python, Java, etc.).
dbo_course['COURSE_SK'] = list(range(1, len(dbo_course['COURSE_id']) + 1))

# Hetzelfde als bij de vorige, maar in plaats van een beschrijving, kan het bijvoorbeeld zijn dat een satisfaction type wordt opgesplitst (zoals Good = Very good, Excellent, etc.).
dbo_satisfaction_type['SATISFACTION_TYPE_SK'] = list(range(1, len(dbo_satisfaction_type['SATISFACTION_TYPE_id']) + 1))


# Exporteer de dataframes naar de SQLite-database
dbo_product.to_sql('PRODUCT', sqlite_conn, if_exists='replace', index=False)
dbo_sales_staff.to_sql('SALES_STAFF', sqlite_conn, if_exists='replace', index=False)
dbo_retailer_contact.to_sql('RETAILER_CONTACT', sqlite_conn, if_exists='replace', index=False)
dbo_returned_reason.to_sql('RETURNED_REASON', sqlite_conn, if_exists='replace', index=False)
dbo_order_method.to_sql('ORDER_METHOD', sqlite_conn, if_exists='replace', index=False)
dbo_course.to_sql('COURSE', sqlite_conn, if_exists='replace', index=False)
dbo_satisfaction_type.to_sql('SATISFACTION_TYPE', sqlite_conn, if_exists='replace', index=False)

Zet alle gegevens om van de SQLite .db bestand naar de SSMS database

In [ ]:
# Lees de data van een tabel uit de SQLite-database
product = pd.read_sql_query('SELECT * FROM PRODUCT', sqlite_conn)
sales_staff = pd.read_sql_query('SELECT * FROM SALES_STAFF', sqlite_conn)
retailer_contact = pd.read_sql_query('SELECT * FROM RETAILER_CONTACT', sqlite_conn)
returned_reason = pd.read_sql_query('SELECT * FROM RETURNED_REASON', sqlite_conn)
order_method = pd.read_sql_query('SELECT * FROM ORDER_METHOD', sqlite_conn)
course = pd.read_sql_query('SELECT * FROM COURSE', sqlite_conn)
satisfaction_type = pd.read_sql_query('SELECT * FROM SATISFACTION_TYPE', sqlite_conn)


# Itereer over de rijen van iedere dataframe en voeg elke record toe aan de SSMS-database

for index, row in product.iterrows():
    try:
        query = f"INSERT INTO dbo.rPRODUCT VALUES ({row['PRODUCT_id']}, '{row['PRODUCT_name']}', '{row['PRODUCT_image']}', '{row['PRODUCT_LINE_name']}', '{row['PRODUCT_TYPE_name']}', '{row['PRODUCT_INTRODUCTION_DATE_date']}', {row['PRODUCT_COST_price']}, {row['PRODUCT_MARGIN_dimensions']}, '{row['PRODUCT_LANGUAGE_name']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in sales_staff.iterrows():
    try:
        query = f"INSERT INTO dbo.rSALES_STAFF VALUES ({row['SALES_STAFF_id']}, '{row['SALES_STAFF_f_name']}', '{row['SALES_STAFF_l_name']}', '{row['SALES_STAFF_phone']}', '{row['SALES_STAFF_email']}', '{row['SALES_STAFF_fax']}', {row['SALES_STAFF_sales_branch_id']}, '{row['SALES_STAFF_POSITION_EN_name']}', '{row['SALES_STAFF_DATE_HIRED_date']}', '{row['SALES_STAFF_EXTENSION_extension']}', '{row['SALES_STAFF_SALES_BRANCH_address1']}', '{row['SALES_STAFF_SALES_BRANCH_address2']}', '{row['SALES_STAFF_SALES_BRANCH_region']}', '{row['SALES_STAFF_SALES_BRANCH_city']}', '{row['SALES_STAFF_SALES_BRANCH_postal_zone']}', {row['SALES_STAFF_COUNTRY_id']}, '{row['SALES_STAFF_COUNTRY_name']}', '{row['SALES_STAFF_COUNTRY_language']}', '{row['SALES_STAFF_COUNTRY_currency']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in retailer_contact.iterrows():
    try:
        query = f"INSERT INTO dbo.RETAILER_CONTACT VALUES ({row['retailer_contact_contact_id']}, {row['retailer_contact_site_id']}, '{row['retailer_contact_first_name']}', '{row['retailer_contact_last_name']}', '{row['retailer_contact_job_position_en']}', {row['retailer_contact_extension']}, {row['retailer_contact_fax']}, '{row['retailer_contact_email']}', '{row['retailer_contact_gender']}', {row['retailer_id']}, {row['retailer_codemr']}, '{row['retailer_company_name']}', {row['retailer_age_group_id']}, {row['retailer_age_group_upper_age']}, {row['retailer_age_group_lower_age']}, {row['retailer_sales_demographic_id']}, {row['retailer_sales_demographic_codemr']}, {row['retailer_sales_demographic_age_group_id']}, {row['retailer_sales_demographic_sales_percent']}, {row['retailer_sales_territory_id']}, '{row['retailer_sales_territory_name_en']}', {row['retailer_country_id']}, '{row['retailer_country_country_en']}', '{row['retailer_country_flag_image']}', {row['retailer_country_sales_territory_id']}, {row['retailer_site_id']}, {row['retailer_site_retailer_id']}, '{row['retailer_site_address1']}', '{row['retailer_site_address2']}', '{row['retailer_site_city']}', '{row['retailer_site_region']}', '{row['retailer_site_postal_zone']}', {row['retailer_site_country_id']}, {row['retailer_site_active_indicator']}, {row['retailer_headquarters_codemr']}, '{row['retailer_headquarters_name']}', '{row['retailer_headquarters_address1']}', '{row['retailer_headquarters_address2']}', '{row['retailer_headquarters_city']}', '{row['retailer_headquarters_region']}', '{row['retailer_headquarters_postal_zone']}', {row['retailer_headquarters_country_id']}, '{row['retailer_headquarters_phone']}', '{row['retailer_headquarters_fax']}', {row['retailer_headquarters_segment_id']}, {row['retailer_type_id']}, '{row['retailer_type_type_en']}', {row['retailer_segment_id']}, '{row['retailer_segment_language']}', '{row['retailer_segment_name']}', '{row['retailer_segment_description']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in returned_reason.iterrows():
    try:
        query = f"INSERT INTO dbo.RETURN_REASON VALUES ({row['RETURNED_REASON_id']}, '{row['RETURNED_REASON_DESCRIPTION_name']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in order_method.iterrows():
    try:
        query = f"INSERT INTO dbo.ORDER_METHOD VALUES ({row['ORDER_METHOD_id']}, '{row['ORDER_METHOD_EN_name']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in course.iterrows():
    try:
        query = f"INSERT INTO dbo.COURSE VALUES ({row['COURSE_id']}, '{row['COURSE_DESCRIPTION_en']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in satisfaction_type.iterrows():
    try:
        query = f"INSERT INTO dbo.SATISFACTION_TYPE VALUES ({row['SATISFACTION_TYPE_id']}, '{row['SATISFACTION_TYPE_DESCRIPTION_name']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

In [ ]:
pd.read_sql_query('SELECT * FROM dbo.rPRODUCT', ssms_conn)

Sluit connecties met alle databases

In [ ]:
sqlite_conn.close()
ssms_conn.close()